In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import nb_general as nb
import scipy.linalg as la
import non_backtracking_tools as nbt
import time
import sys
import sympy as sy

In [2]:
# Make graph
G = nx.Graph([[1,2],[2,3],[3,1],[3,4],[2,4]])
# make K
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

In [3]:
mu_vals, mu_vecs = la.eig(K)
lambda_vals, lambda_vecs = la.eig(A)

In [4]:
mu_vecs[:,np.argsort(mu_vals)[-3]]

array([-7.99024334e-16+5.00000000e-01j, -6.08416932e-17-8.66203294e-17j,
        1.91595179e-17+4.91971467e-17j, -1.60589429e-19-5.00000000e-01j,
       -5.00000000e-01-8.73305282e-16j,  2.32265345e-16-7.86714394e-17j,
        4.93076885e-17+3.93225472e-17j,  5.00000000e-01+0.00000000e+00j])

In [5]:
# Calculate with formuula
formula = np.zeros(lambda_vals.shape[0]*2)
for i in range(0,len(formula),2):
    x = lambda_vecs[:,i//2]
    y2 = mu_vecs[mu_vecs.shape[0]//2:,0]
    y1 = mu_vecs[mu_vecs.shape[0]//2:,i+1]
    formula[i] = (lambda_vals[i//2]*(x.T@y1) + 
                  np.sqrt((lambda_vals[i//2]*(x.T@y1))**2
                          -4*x.T@(D-np.eye(D.shape[0]))@y1))/2
#     formula[i+1] = (lambda_vals[i//2]*(x.T@y2) - np.sqrt((lambda_vals[i//2]*(x.T@y2))**2-4*x.T@(D-np.eye(D.shape[0]))@y1))/2
print(formula)

[8.98860183e-01 0.00000000e+00 1.21376870e-08 0.00000000e+00
 1.69842559e-08 0.00000000e+00 9.77411867e-01 0.00000000e+00]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


In [6]:
# Get all possible y
y = np.zeros((lambda_vecs.shape[0],mu_vecs.shape[1]))
for i in range(mu_vecs.shape[1]):
    y[:,i] = mu_vecs[mu_vecs.shape[0]//2:,i]
print(y)

[[ 2.62651044e-01 -3.53553391e-01  2.55288980e-01  2.55288980e-01
  -5.00000000e-01 -5.00000000e-01 -1.13985294e-16 -1.13985294e-16]
 [ 2.86116001e-01 -3.53553391e-01 -2.00512274e-01 -2.00512274e-01
   2.32265345e-16  2.32265345e-16  1.44337567e-01  1.44337567e-01]
 [ 2.86116001e-01 -3.53553391e-01 -2.00512274e-01 -2.00512274e-01
   4.93076885e-17  4.93076885e-17 -1.44337567e-01 -1.44337567e-01]
 [ 2.62651044e-01 -3.53553391e-01  2.55288980e-01  2.55288980e-01
   5.00000000e-01  5.00000000e-01 -1.34906774e-16 -1.34906774e-16]]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


In [7]:
possible = []
for i in range(y.shape[1]):
    x = lambda_vecs[:,0]
    possible.append((lambda_vals[0]*(x.T@y[:,i])+np.sqrt((lambda_vals[0]*(x.T@y[:,i]))**2-4*x.T@(D-np.eye(D.shape[0]))@y[:,i])/2))

In [8]:
np.real(lambda_vals)

array([ 2.56155281e+00,  8.95246063e-17, -1.56155281e+00, -1.00000000e+00])

In [9]:
np.real(possible)

array([-2.37084904e-01,  1.79772037e+00,  3.39403213e-03,  3.39403213e-03,
        1.97123827e-08,  1.97123827e-08,  3.19938178e-16,  3.19938178e-16])

In [10]:
A = np.array([[0,1,0,1],[1,0,1,1],[0,1,0,0],[1,1,0,0]])
la.eig(A)

(array([ 2.17008649+0.j,  0.31110782+0.j, -1.        +0.j, -1.4811943 +0.j]),
 array([[ 5.22720726e-01,  3.68160356e-01,  7.07106781e-01,
          3.02028137e-01],
        [ 6.11628457e-01, -2.53622791e-01,  5.95213736e-16,
         -7.49390492e-01],
        [ 2.81845199e-01, -8.15224745e-01, -4.58144079e-16,
          5.05936655e-01],
        [ 5.22720726e-01,  3.68160356e-01, -7.07106781e-01,
          3.02028137e-01]]))

In [15]:
G = nx.Graph([[1,2],[2,3],[2,4],[3,4]])
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

In [16]:
# Get eigenvalues
lambda_vals, lambda_vecs = la.eig(A)
mu_vals, mu_vecs = la.eig(K)

In [17]:
lambda_2 = lambda_vals[1]

In [18]:
mu_2 = mu_vals[3]

In [19]:
x = lambda_vecs[:,1]
y = mu_vecs[4:,3]

In [20]:
c = x.T@y
d = x.T@(D-np.eye(D.shape[0]))@y

In [21]:
(lambda_2*c+np.sqrt((lambda_2*c)**2-4*d*c))/(2*c)

(-0.6888921512261111-0j)

### Odd that it works with the negative

In [22]:
(lambda_2*c-np.sqrt((lambda_2*c)**2-4*d*c))/(2*c)

(0.999999968692093-0j)

In [23]:
mu_2

(0.999999968692088+0j)

In [24]:
mu_vals

array([ 0.00000000e+00+0.j       , -5.00000000e-01+0.8660254j,
       -5.00000000e-01-0.8660254j,  9.99999969e-01+0.j       ,
        1.00000003e+00+0.j       , -5.00000000e-01+0.8660254j,
       -5.00000000e-01-0.8660254j, -7.77193096e-17+0.j       ])

In [25]:
# Check formula multiple times
positive = []
negative = []
neither = []
avg_mu_bound = []
min_mu_bound = []
simple_upper = []
simple_lower = []
rho_bound = []

# Create progress bar
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1))

# Create 50 random graphs
for i in range(50):
    # Create random graph that is connected
    G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)    # Create adjacency matrix
    
    # Create adjacency matrix, D matrix, D^{-1} matrix
    A = nx.adjacency_matrix(G).todense()
    D = np.diag(list(dict(G.degree).values()))
    D_inv = np.diag([1/i for i in dict(G.degree).values()])
    
    # Create K
    K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])
    
    # Get eigenvalues and eigenvectors of adjacecny matrix and K matrix
    lambda_vals, lambda_vecs = la.eig(A)
    mu_vals, mu_vecs = la.eig(K)
    
    # Find the second largest eigenvalue of K
    idx = np.argsort(mu_vals)[-2]
    mu_2 = mu_vals[idx]
    
    # Find y
    y = mu_vecs[mu_vecs.shape[0]//2:,idx]
    
    # Find second largest eigenvalue of A
    idx = np.argsort(lambda_vals)[-2]
    lambda_2 = lambda_vals[idx]
    
    # Find x
    x = lambda_vecs[:,idx]
    
    # Calculate x^Ty and x^T(D-I)y
    c = x.T@y
    d = x.T@(D-np.eye(D.shape[0]))@y
    
    # Calculate average, minimum, and maximum degree
    d_bar = np.mean(list(dict(G.degree).values()))
    d_min = np.min(list(dict(G.degree).values()))
    d_max = np.max(list(dict(G.degree).values()))
    
    # Check whether mu_2 is calculated with positive version of formula
    if la.norm(np.abs(mu_2)-np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*c*d))/(2*c))) <  1e-8:
        positive.append(G)
        
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_bar
        if np.abs(mu_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c)):
            avg_mu_bound.append(mu_2)
            
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_min    
        if np.abs(mu_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_min-1)))/(2*c)):
            min_mu_bound.append(mu_2)
        
        # If mu_2 is not bound by replacing x^T(D-I)y with d_min, look at the value of mu_2
        else:
            print(mu_2)
            
    # Check whether mu_2 is calculated with negative version of formula
    elif la.norm(np.abs(mu_2)-np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*c*d))/(2*c))) <  1e-8:
        negative.append(G)
        
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_bar
        if np.abs(mu_2) <= np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c)):
            avg_mu_bound.append(mu_2)
            
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_min
        if np.abs(mu_2) <= np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*(d_min-1)))/(2*c)):
            min_mu_bound.append(mu_2)
            
        # If mu_2 is not bound by replacing x^T(D-I)y with d_min, look at the value of mu_2
        else:
            print(mu_2)
    
    # Collect if mu_2 is not calculated
    else:
        neither.append(G)
    
    # Get eigenvalues and eigenvectors of non-backtracking P
#     rho_vals, rho_vecs = la.eig(nbt.trans_prob_matrix_nb(G))
    
#     # Get second largest of non-backtracking P
#     idx = np.argsort(rho_vals)[-2]
#     rho_2 = rho_vals[idx]
    
    # Check whether mu_2 <= lambda_2/d_min
    if np.abs(mu_2) <= np.abs(lambda_2)/d_min:
        simple_upper.append(G)
        
    # Check whether mu_2 >= lambda_2/d_max
    if np.abs(mu_2) >= np.abs(lambda_2)/d_max:
        simple_lower.append(G)
        
#     # Check whether rho_2 is bound by using the average degree and formula
#     if np.abs(rho_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c*(d_bar-1))):
#         rho_bound.append(G)
        
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("]\n") # this ends the progress bar

[                                        --------------------------------------------------]


In [26]:
len(positive)

24

In [27]:
len(negative)

26

In [28]:
len(neither)

0

In [29]:
len(avg_mu_bound)

50

In [30]:
len(min_mu_bound)

50

In [31]:
len(simple_lower)

50

In [32]:
len(simple_upper)

1

In [33]:
len(rho_bound)

0

In [34]:
min_mu_bound

[(12.425577762977984+0j),
 (15.18230071464303+0j),
 (9.20536611810048+0j),
 (2.172832459492344+0.5428685119695643j),
 (15.301298429086577+0j),
 (17.50954796543063+0j),
 (5.187867092859528+0j),
 (5.260921558510316+0j),
 (12.96801190702124+0j),
 (14.119660731592564+0j),
 (5.893205592323404+0j),
 (4.019854630984682+0j),
 (3.4455651777+3.0791392417878694j),
 (4.550654605338831+0j),
 (5.345878340384699+0j),
 (8.763174976983858+0j),
 (3.623123948695738+3.2499326377666895j),
 (5.5567027762305905+0j),
 (13.121977365129196+0j),
 (14.643009494100756+0j),
 (7.134204737462263+0j),
 (4.065729875537525+2.5529261692546603j),
 (5.400414885741358+0j),
 (12.804722101109972+0j),
 (7.017733640495727+0j),
 (15.519376436890331+0j),
 (10.19011032442227+0j),
 (4.354665437994208+0j),
 (10.555738843849603+0j),
 (7.430795301023855+0j),
 (13.236379431474623+0j),
 (11.32640870752055+0j),
 (7.536109927199891+0j),
 (10.941014490483624+0j),
 (6.688871002541232+0j),
 (3.8510593334773193+0j),
 (7.401845523955801+0j),
 

In [35]:
# Check that K is diagonalizable
sy.Matrix(K).is_diagonalizable()

KeyboardInterrupt: 

In [ ]:
for i in positive:
    print(f'{len(i.degree)}\t{len(i.edges)}')

In [ ]:
for i in negative:
    print(f'{len(i.degree)}\t{len(i.edges)}')

In [ ]:
ortho = dict()
# Create random graph that is connected
G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)    # Create adjacency matrix
# Create adjacency matrix, D matrix, D^{-1} matrix
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))

# Create K
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

# Calculate all y
mu_vals, mu_vecs = la.eig(K)
Y = mu_vecs[mu_vecs.shape[0]//2:,:]

lambda_vals, lambda_vecs = la.eig(A)
for i in range(len(lambda_vals)):
    vecs = []
    for j in range(len(mu_vals)):
#         print(lambda_vecs[:,i].T@Y[:,j])
        if la.norm(lambda_vecs[:,i].T@Y[:,j]) < 1e-5:
            vecs.append(j)
    ortho[i] = vecs

In [ ]:
# G = nx.Graph([[1,2],[2,3],[3,4],[4,5],[2,5],[3,5],[5,6],[6,3]])
G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)
F = list(nx.connected_component_subgraphs(G))
G = F[0]
# Create adjacency matrix, D matrix, D^{-1} matrix
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))

# Create K
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

# Calculate all y
mu_vals, mu_vecs = la.eig(K)
Y = mu_vecs[mu_vecs.shape[0]//2:,:]
lambda_vals, lambda_vecs = la.eig(A)

for i in range(len(mu_vals)):
    # Give current mu
    print(f'$\\mu_{i}$: {mu_vals[i]}')
    for k in range(len(lambda_vals)):
        # Check for orthogonality
        if lambda_vecs[:,k].T@Y[:,i] < 1e-8 and lambda_vecs[:,k].T@Y[:,i] > -1e-8:
            print(f"\t$\\lambda_{k}$: Orthogonal")
            continue
        c = lambda_vecs[:,k].T@Y[:,i]
        d = lambda_vecs[:,k].T@(D-np.eye(D.shape[0]))@Y[:,i]
        positive = (lambda_vals[k]*c+np.sqrt((lambda_vals[k]*c)**2-4*c*d))/(2*c)
        negative = (lambda_vals[k]*c-np.sqrt((lambda_vals[k]*c)**2-4*c*d))/(2*c) 
        # Check for positive version
        print(f'\t$\\lambda_{k}$ (positive): {positive}')
        # Check for negative version
        print(f'\t$\\lambda_{k}$ (negative): {negative}')

In [ ]:
for _ in range(int(100)):
    G = nx.random_partition_graph([5,5],0.25,.05)
    F = list(nx.connected_component_subgraphs(G))
    G = F[0]
    # Create adjacency matrix, D matrix, D^{-1} matrix
    A = nx.adjacency_matrix(G).todense()
    D = np.diag(list(dict(G.degree).values()))

    # Create K
    K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

    # Calculate all y
    mu_vals, mu_vecs = la.eig(K)
    Y = mu_vecs[mu_vecs.shape[0]//2:,:]
    lambda_vals, lambda_vecs = la.eig(A)
    for i in range(len(mu_vals)):
        # Give current mu
        count = 0
        for k in range(len(lambda_vals)):
            # Check for orthogonality
            if lambda_vecs[:,k].T@Y[:,i] < 1e-8 and lambda_vecs[:,k].T@Y[:,i] > 1e-8:
                print(f'$\mu_{i}$')
        

In [ ]:
D

In [45]:
G = nx.Graph([[1,2],[2,3],[3,4],[2,4],[4,5],[3,5],[5,1],[5,2],[5,3]])
C, B = nb.to_edge_space(G,B=True,graph=False)
A = nx.adjacency_matrix(G).todense()

In [46]:
la.eig(B)[0]

array([ 2.27499334+0.j        , -0.5       +1.6583124j ,
       -0.5       -1.6583124j ,  0.12899486+1.16227771j,
        0.12899486-1.16227771j, -0.76649153+1.15804008j,
       -0.76649153-1.15804008j,  1.        +0.j        ,
        1.        +0.j        , -0.5       +1.32287566j,
       -0.5       -1.32287566j, -1.        +0.j        ,
       -1.        +0.j        , -1.        +0.j        ,
        1.        +0.j        ,  1.        +0.j        ])

In [49]:
formula = lambda x, vec: pass

SyntaxError: invalid syntax (<ipython-input-49-b8d08b6523c4>, line 1)